In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("mai_clients.csv")
df

,ShopId,Datetime,OrderId,EmployeeId,Total,nunique_item_per_order,employee_total_n_orders,employee_total_money,Total_money
0,4,2021-01-04 06:24:33,6386379,28,4.35,3,642,2118.19,21785.55
1,4,2021-01-04 06:27:12,6386383,28,3.10,2,642,2118.19,21785.55
2,4,2021-01-04 06:31:18,6386391,28,2.40,2,642,2118.19,21785.55
3,4,2021-01-04 06:32:34,6386392,28,3.40,2,642,2118.19,21785.55
4,4,2021-01-04 06:33:21,6386395,28,2.55,1,642,2118.19,21785.55
...,...,...,...,...,...,...,...,...,...
10936,5,2021-01-30 17:29:53,6440694,206,3.00,1,189,1208.30,31872.42
10937,5,2021-01-30 17:32:48,6440698,209,7.00,1,505,3432.21,31872.42
10938,5,2021-01-30 17:36:38,6440706,206,7.80,3,189,1208.30,31872.42
10939,5,2021-01-30 17:44:52,6440715,209,8.45,3,505,3432.21,31872.42


## Lab1

In [5]:
df = pd.read_csv("mai_clients.csv")
df.Datetime = df.Datetime.apply(pd.to_datetime)
df.drop(
    ["employee_total_n_orders", "OrderId", "nunique_item_per_order"],
    axis=1,
    inplace=True,
)
TimeBetweenOrders = (
    df.groupby(by=["ShopId", "EmployeeId"])
    .Datetime.apply(lambda x: x[1:] - x.shift(1).dropna())
    .reset_index(drop=True)
)
df_time = pd.merge(df, TimeBetweenOrders, how="left", left_index=True, right_index=True)
df_time = df_time.rename(columns={"Datetime_y": "ServiceTime"}, inplace=False)
df_time.ServiceTime = df_time.ServiceTime.apply(lambda x: x.total_seconds())
Employee_stats = df_time.groupby(by=["ShopId", "EmployeeId"]).aggregate("median")
Shops_columns = {"Total_money"}
Employee_criterions = set(Employee_stats.columns).difference(Shops_columns)
Employee_criterions = list(Employee_criterions)



In [6]:
# Consider worker's best result for comparison
lookup = Employee_stats[Employee_criterions].reset_index().groupby(by=["EmployeeId"])
lookup = (
    pd.concat(
        [lookup[criterion].apply(sum) for criterion in Employee_criterions], axis=1
    )
    .reset_index()
    .groupby(by=["EmployeeId"])
    .head(1)
)

In [7]:
rankets = pd.concat(
    [lookup.EmployeeId]
    + [
        lookup[criterion].rank(
            method="dense", ascending=False if criterion != "ServiceTime" else True
        )
        for criterion in Employee_criterions
    ],
    axis=1,
).set_index("EmployeeId")

rankets.columns = ["rank " + criterion for criterion in Employee_criterions]
Employee_stats_ranked = pd.merge(
    Employee_stats.reset_index(), rankets, left_on=["EmployeeId"], right_index=True
).set_index(["ShopId", "EmployeeId"])


In [8]:
# order = [Total	empol_total_money	ServiceTime]
D = np.array(
    [[1, 7, 3], [1/7, 1, 1/7], [1/3, 7, 1]]
)

In [9]:
def EM(D):
    n = D.shape[0]
    def MRCI(n):
        table1 = [0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.40, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]
        if n >= len(table1):
            raise ValueError("n too chonky")
        return table1[n-1]

    def get_CR(max_lambda):
        return ((max_lambda - n) / (n - 1)) / MRCI(n)

    w,v = np.linalg.eig(D)
    lambda_max = np.max(w).astype(np.float32)
    weights = v[:,np.argmax(w)].astype(np.float32)
    return {"w": weights, "CR": get_CR(lambda_max)}


em_res = EM(D)
local_weights = em_res["w"]
local_weights, em_res["CR"]

<ipython-input-9-01c701a33714>:13: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_max = np.max(w).astype(np.float32)
<ipython-input-9-01c701a33714>:14: ComplexWarning: Casting complex values to real discards the imaginary part
  weights = v[:,np.argmax(w)].astype(np.float32)


(array([0.89768934, 0.08891758, 0.43156403], dtype=float32),
 0.1303950181374183)

In [10]:
def get_Employee_weights(rankets, local_weights, aggregation_method, order=True):

    rankets_inv = rankets.applymap(lambda x: 1.0 / x) if order else rankets
    rankets_inv_normalized = rankets_inv.divide(rankets_inv.sum(axis=1), axis=0)

    if aggregation_method == "distributive":

        rankets_inv_normalized_weighted = rankets_inv_normalized.multiply(
            local_weights, axis=1
        )
        result = rankets_inv_normalized_weighted.sum(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)

    elif aggregation_method == "min":

        rankets_inv_normalized_weighted = rankets_inv_normalized.multiply(
            local_weights, axis=1
        )
        result = rankets_inv_normalized_weighted.min(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)

    elif aggregation_method == "geom":

        rankets_inv_normalized_weighted = rankets_inv_normalized.pow(
            local_weights, axis=1
        )
        result = rankets_inv_normalized_weighted.prod(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)

    elif aggregation_method == "ideal_point":

        rankets_inv_normalized_weighted = rankets_inv_normalized.divide(
            rankets_inv_normalized.max(axis=1), axis=0
        )
        result = rankets_inv_normalized_weighted.sum(axis=1)
        result_norm = (result / result.sum()).sort_values(ascending=False)

    return result_norm

methods = ('distributive', 'ideal_point', 'geom', 'min')
Employee_ranked = pd.concat([pd.Series(get_Employee_weights(rankets, local_weights, method).index) \
           for method in methods], axis=1)
Employee_ranked.columns = methods
Employee_ranked[['distributive']]

,distributive
0,4
1,192
2,100
3,15
4,209
5,18
6,161
7,70
8,28
9,7


In [11]:
test = (
    Employee_stats_ranked.reset_index()
    .groupby("EmployeeId")
    .apply(
        lambda x: x[
            (list(Employee_criterions))
            + ["rank Total", "rank employee_total_money", "rank ServiceTime"]
        ].sort_values("employee_total_money", ascending=False)
    )
)

pd.merge(
    test.reset_index(),
    Employee_ranked[["distributive"]].reset_index(),
    left_on="EmployeeId",
    right_on="distributive",
).sort_values("index", ascending=True)


,EmployeeId,level_1,Total,ServiceTime,employee_total_money,rank Total,rank employee_total_money,rank ServiceTime,index,distributive
0,4,1,5.050,434.5,481.36,1.0,16.0,15.0,0,4
1,4,0,444.000,26.0,444.00,1.0,16.0,15.0,0,4
26,192,16,4.300,69.0,22.45,2.0,13.0,20.0,1,192
25,192,17,5.800,483.0,1687.75,2.0,13.0,20.0,1,192
17,100,29,8.750,701.5,62.60,4.0,24.0,24.0,2,100
6,15,3,6.250,357.0,391.11,3.0,20.0,9.0,3,15
7,15,2,2.550,56.0,28.75,3.0,20.0,9.0,3,15
32,209,22,5.300,476.0,3432.21,5.0,7.0,23.0,4,209
33,209,21,2.825,206.5,66.80,5.0,7.0,23.0,4,209
9,18,4,2.675,62.0,114.05,6.0,11.0,17.0,5,18


In [12]:
test = (
    Employee_stats_ranked.reset_index()
    .groupby("EmployeeId")
    .head(5)
    .sort_values("employee_total_money", ascending=False)
)
test[test.ShopId == 5]

,ShopId,EmployeeId,Total,employee_total_money,Total_money,ServiceTime,rank Total,rank ServiceTime,rank employee_total_money
24,5,10,5.600,3643.76,31872.42,416.0,15.0,10.0,5.0
26,5,14,4.900,3609.42,31872.42,456.0,18.0,14.0,6.0
22,5,209,5.300,3432.21,31872.42,476.0,5.0,23.0,7.0
12,5,161,4.800,3202.64,31872.42,413.0,7.0,19.0,8.0
15,5,176,4.800,2785.21,31872.42,347.5,9.0,12.0,3.0
30,5,144,5.000,2458.54,31872.42,381.0,17.0,7.0,10.0
5,5,18,5.350,2202.83,31872.42,430.0,6.0,17.0,11.0
25,5,13,6.975,1858.13,31872.42,446.0,13.0,13.0,12.0
17,5,192,5.800,1687.75,31872.42,483.0,2.0,20.0,13.0
23,5,7,7.350,1466.45,31872.42,507.0,11.0,18.0,14.0


In [13]:
Employee_stats_ranked

Total  employee_total_money  Total_money  ServiceTime  \
ShopId EmployeeId                                                            
4      4           444.000                444.00     21785.55         26.0   
5      4             5.050                481.36     31872.42        434.5   
4      15            2.550                 28.75     21785.55         56.0   
5      15            6.250                391.11     31872.42        357.0   
4      18            2.675                114.05     21785.55         62.0   
5      18            5.350               2202.83     31872.42        430.0   
4      21            2.800                505.00     21785.55         79.0   
       28            2.600               2118.19     21785.55        113.0   
5      28            4.825                772.46     31872.42        517.0   
4      30            2.575                589.84     21785.55         77.0   
       98            2.950                 38.50     21785.55         51.0   
       161           3.200                 25.00     21785.55        135.0   
5      161           4.800               3202.64     31872.42        413.0   
4      173           2.750               5569.18     21785.55        102.0   
       176           2.750               2287.68     21785.55         98.0   
5      176           4.800               2785.21     31872.42        347.5   
4      192           4.300                 22.45     21785.55         69.0   
5      192           5.800               1687.75     31872.42        483.0   
4      196           2.900               2870.65     21785.55         85.0   
5      196           5.000               1107.91     31872.42        407.0   
4      205           2.750               7105.46     21785.55         96.0   
       209           2.825                 66.80     21785.55        206.5   
5      209           5.300               3432.21     31872.42        476.0   
       7             7.350               1466.45     31872.42        507.0   
       10            5.600               3643.76     31872.42        416.0   
       13            6.975               1858.13     31872.42        446.0   
       14            4.900               3609.42     31872.42        456.0   
       29            4.500                332.97     31872.42        428.5   
       70            7.100                388.21     31872.42        664.5   
       100           8.750                 62.60     31872.42        701.5   
       144           5.000               2458.54     31872.42        381.0   
       187           4.300                465.98     31872.42        491.0   
       206           5.150               1208.30     31872.42        412.0   
       207           5.950                314.58     31872.42        310.0   

                   rank Total  rank ServiceTime  rank employee_total_money  
ShopId EmployeeId                                                           
4      4                  1.0              15.0                       16.0  
5      4                  1.0              15.0                       16.0  
4      15                 3.0               9.0                       20.0  
5      15                 3.0               9.0                       20.0  
4      18                 6.0              17.0                       11.0  
5      18                 6.0              17.0                       11.0  
4      21                22.0               3.0                       18.0  
       28                10.0              21.0                        9.0  
5      28                10.0              21.0                        9.0  
4      30                24.0               2.0                       17.0  
       98                21.0               1.0                       25.0  
       161                7.0              19.0                        8.0  
5      161                7.0              19.0                        8.0  
4      173               23.0               5.0          